<div style="background-color: #f4f4f4; color: #333333; padding: 20px; border-radius: 10px; box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1); max-width: 2500px; margin: auto;">

<h1 style="color: #347a87;">COMM475</h1>
<h2 style="color: #d97706;">Investment Policies</h2>
<h3 style="color: #316cbe;">Replicating CRR (1986)</h3>

<p style="margin-bottom: 0;"><strong>Instructor:</strong> Lorenzo Garlappi © 2024*</p>
<p><strong>TAs:</strong> Gen Li, Martin Rinaldi, and Tianping Wu </p>

<p style="color: #a85d32;">* These notes are based on Rob Heinkel and Bill Tilford's lecture notes. I am grateful to them for generously sharing this material with me.</p>

</div>

# Part I: Housekeeping

<div style="background-color: #f4f4f4; color: #333333; padding: 20px; border-radius: 10px; box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1); max-width: 2500px; margin: auto;">

<p><strong>In the following, we provide the example code to replicate CRR (1986) estimation of the risk premium.</strong></p>

<p><strong>We first import the basic packages we need to run the exercise: </strong></p>

</div>

In [42]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm 

from datetime import datetime
from IPython.display import display

import certifi
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

# Ignore harmless warnings
import warnings
warnings.filterwarnings("ignore")

## Data loading and cleaning

In [43]:
url = 'https://raw.githubusercontent.com/lorenzogarlappi/COMM475/main/Data/20_portfolios_size_ew_Rfree.csv'
Assets = pd.read_csv(url)

url = 'https://raw.githubusercontent.com/lorenzogarlappi/COMM475/main/Data/Macro.csv'
Macro = pd.read_csv(url)

Macro = Macro[["DATE", "MP", "UPR", "UTS", "UI", "DEI"]]

In [44]:
# Convert DATE to datetime and adjust MACRO_PMF DATE
Macro['DATE'] = pd.to_datetime(Macro['DATE']) 
Assets['DATE'] = pd.to_datetime(Assets['DATE']) + pd.offsets.MonthEnd(0)

<div style="background-color: #f4f4f4; color: #333333; padding: 20px; border-radius: 10px; box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1); max-width: 2500px; margin: auto;">

<p><strong>Now we are ready to merge data in both dataframes and set the column <code>DATE</code> as the index, so the program understands that we are dealing with time series.</strong></p>

</div>

In [45]:
#Left join the data_df with Macro_PMF
Assets_and_Macro = pd.merge(Assets, Macro, on=['DATE'], how='left')

# Set the 'Date' column as the index
Assets_and_Macro.set_index('DATE', inplace=True)

In [46]:
Assets_and_Macro.head()

,q5,q10,q15,q20,q25,q30,q35,q40,q45,q50,...,q85,q90,q95,q100,Rfree,MP,UPR,UTS,UI,DEI
DATE,,,,,,,,,,,,,,,,,,,,,
1941-02-28,0.337954,0.002827,0.006911,-0.009922,-0.001050,-0.006356,-0.018043,-0.016556,-0.027938,-0.018750,...,-0.028010,-0.026722,-0.038421,-0.040107,-0.0001,0.029136,-0.0014,0.0018,NaN,NaN
1941-03-31,-0.003754,0.001573,-0.049657,-0.032795,-0.032885,-0.033706,-0.019767,-0.023764,-0.018991,-0.024744,...,-0.014598,-0.021164,-0.016582,-0.010318,0.0001,0.030856,-0.0118,0.0092,NaN,NaN
1941-04-30,0.000290,0.036065,0.044239,0.018883,0.011086,0.029361,0.027652,0.022785,0.014577,0.012616,...,0.024256,0.015471,-0.006379,0.013832,-0.0001,0.002520,-0.0051,0.0118,NaN,NaN
1941-05-31,-0.027821,-0.072414,-0.075975,-0.065273,-0.051783,-0.078402,-0.057542,-0.050667,-0.078842,-0.048788,...,-0.047524,-0.045722,-0.038033,-0.050276,0.0000,0.044455,0.0022,0.0017,NaN,NaN
1941-06-30,-0.036010,0.042411,-0.004611,0.010849,0.014771,0.016349,0.014421,0.009208,0.019127,0.009969,...,-0.006952,0.014550,0.006699,0.009026,0.0000,0.007224,-0.0003,0.0055,NaN,NaN


<div style="background-color: #f4f4f4; color: #333333; padding: 20px; border-radius: 10px; box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1); max-width: 2500px; margin: auto;">

### Sanity Check: Verifying Correlation with CRR (1986)

Let's conduct our first sanity check. We'll compare the correlation between economic variables in our dataset to the correlations presented in Table 2 of CRR (1986).

</div>

In [54]:
# Instructions for setting the date range
print("To replicate Table 2 of the paper, set start_date to '1952-12-31' and end_date to '1984-01-31'.")
print("Earliest allowed date: '1947-04-30'. Latest allowed date: '2022-12-31'.")

# Select the date range here
start_date = '1952-12-31'  # Earliest allowed date: '1947-04-30'
end_date = '2022-12-31'    # Latest allowed date: '2022-12-31'

# Convert the date strings to datetime objects
begin_date_dt = pd.to_datetime(start_date)
end_date_dt = pd.to_datetime(end_date)

# Check if the dates are within the allowed range
if begin_date_dt < pd.to_datetime('1947-04-30') or end_date_dt > pd.to_datetime('2022-12-31'):
    raise ValueError("The selected dates are out of the allowed range.")

# Now, let's filter our data based on the selected date range
filtered_data = Assets_and_Macro.loc[begin_date_dt:end_date_dt]

# Select variables for the correlation matrix
selected_vars = ['MP', 'UPR', 'UTS', 'UI', 'DEI']

# Calculate and display the correlation matrix in a lower triangular form
correlation = filtered_data[selected_vars].corr(method='spearman').round(3)
mask = np.triu(np.ones_like(correlation, dtype=bool))
lower_triangular_correlation = correlation.mask(mask)
formatted_correlation = lower_triangular_correlation.style.format(lambda x: '' if pd.isnull(x) else f'{x:.3f}'.lstrip('0'))
display(formatted_correlation)

# Print a note to encourage exploration
print("Feel free to adjust the begin_date and end_date variables to explore different periods and observe how correlations might change. Note that the range you pick here will affect the rest of the results in this notebook.")

To replicate Table 2 of the paper, set start_date to '1952-12-31' and end_date to '1984-01-31'.
Earliest allowed date: '1947-04-30'. Latest allowed date: '2022-12-31'.


,MP,UPR,UTS,UI,DEI
MP,,,,,
UPR,.086,,,,
UTS,-0.047,-0.316,,,
UI,-0.000,.084,-0.052,,
DEI,.078,.108,-0.095,.755,


Feel free to adjust the begin_date and end_date variables to explore different periods and observe how correlations might change. Note that the range you pick here will affect the rest of the results in this notebook.


<div style="background-color: #f4f4f4; color: #333333; padding: 20px; border-radius: 10px; box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1); max-width: 2500px; margin: auto;">

<p style="color: #316cbe; border-bottom: 2px solid #316cbe;"><strong>Part I Recap</strong></p>

Before we move on, let's recap what have we done so far:

1. We imported the packages needed to handle data and run regressions.
2. We loaded two dataframes:
   * `Assets`, that contains monthly returns for 20 equally weighted portfolios. Recall that in CRR (1986) stocks are grouped into portfolios formed based on firm size. Firm size is known to be strongly related to average return (see Banz 1981).
   * `Macro`, that has the monthly time series of the economic factors considered by CRR (1986).
3. We did some standard data cleaning and merged both dataframes into one: `Assets_and_Macro`.
4. We checked our results against CRR (1986) Table 2 and mostly matched, except MP and UTS were not negatively correlated as expected. This difference probably comes from us using data from the Federal Reserve Bank of St. Louis, unlike the *Survey of Current Business* used by CRR (1986).

<p style="color: #a85d32; border-bottom: 2px solid #a85d32;"><strong>Goal </strong></p>

In Part II below, we aim to <span style="border-bottom: 2px dashed #d97706; color: #d97706;">estimate the compensation <em>$\lambda_t^f$</em></span> for exposure to economic state variables, <span style="border-bottom: 2px dashed #d97706; color: #d97706;"><em>$\beta_{i, t}^f$</em></span>. In essence, we are focused on measuring the <strong><span style="color: #d97706;">risk premium</span></strong> — the compensation investors require for bearing risk associated with economic factors likely to affect stock prices.

<p style="color: #a85d32; border-bottom: 2px solid #a85d32;"><strong>Factors</strong></p>

CRR (1986) propose a set of economic state variables that systematically affect stock prices, including <span style="color: #347a87;">innovations to inflation</span>, <span style="color: #347a87;">interest rates</span>, and <span style="color: #347a87;">industrial production</span>.

We will now move on to Part II, were we will follow a series of steps directed to estimate risk premia a la CRR (1986)–in essence, the authors follow the methodology in Fama-MacBeth(1973).

</div>

# Part II: Estimating Risk Premia

<div style="background-color: #f4f4f4; color: #333333; padding: 20px; border-radius: 10px; box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1); max-width: 2500px; margin: auto;">

## Step A: Estimate the assets' exposure to the economic state variables

We will now regress the returns of each equally-weighted size-sorted portfolio on the *unanticipated* changes in the economic variables over the previous 5 years. This is the estimation window used in CRR (1986).  

In other words, in this step we want to use the previous 60 months of data to estimate factor sensitivities, $\beta_{j p}$ under the specification: 

$$
r_{p}=\alpha_{p}+I_{1} \beta_{1 p}+I_{2} \beta_{2 p}+I_{3} \beta_{3 p}+I_{4} \beta_{4 p}+I_{5} \beta_{5 p} + e_{s}
$$

This accounts for a total of 20 regressions (one per each portfolio $p$), each with 60 observations. This gives estimates of $\beta_{j p}$ for $j=1,5$ and $p=1,20$ for each month. 


</div>

### Step A.1: Demeaning Economic State Variables and Computing Excess Returns 

First we will demean the economic factors to map them into *unanticipated* changes.

In [55]:
# Now filter the Assets_and_Macro dataframe based on the selected date range at the beginning of the notebook
Assets_and_Macro_CRR = Assets_and_Macro.loc[start_date:end_date]

# Demean the factors to get the *unanticipated changes* in the factors
factors = ['MP', 'UPR', 'UTS', 'UI', 'DEI']
for factor in factors:
    Assets_and_Macro_CRR[factor] = Assets_and_Macro_CRR[factor] - Assets_and_Macro_CRR[factor].mean()

# For all sorted portfolios q5,q10,...,q100, calculate monthly excess returns by subtracting Rfree
for i in range(5, 105, 5):
    Assets_and_Macro_CRR[f'q{i}'] = Assets_and_Macro_CRR[f'q{i}'] - Assets_and_Macro_CRR['Rfree']

### Step A.2: Estimate $\beta_{j p}$, for $j=1,5$ and $p=1,20$, for all years in the CRR(1986) Table 4 sample 

To make our life easier, let's define a function that takes care of the betas estimation using data from the previous 5 years. We will then delimit the time period observed to match the sample in CRR (1986) and get our estimates $\hat{\beta_{j p}}$, for $j=1,5$ and $p=1,20$. 

In [56]:
def estimate_betas(data, start_date, end_date):
    # Convert the dates you picked to datetime objects
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    
    # Filter the data based on the selected range
    data_filtered = data.loc[start_date:end_date]
    
    # Creating a structure to hold beta estimates for each portfolio and year
    beta_estimates = {f'q{q}': {year: None for year in range(start_date.year, end_date.year)} for q in range(5, 105, 5)}
    
    # Iterating over each year where beta estimation is possible
    for year in range(start_date.year, end_date.year):
        start_date_for_year = pd.to_datetime(f"{year-5}-01-31")  # 5 years before the target year
        end_date_for_year = pd.to_datetime(f"{year-1}-12-31")  # End of the year before the target year
        
        # Now we want to check if the selected window is within the available data range. If not, we skip this iteration.
        if start_date_for_year < data_filtered.index.min() or end_date_for_year > data_filtered.index.max():
            continue
        
        # Select the window for the current iteration
        window = data_filtered.loc[start_date_for_year:end_date_for_year]
        
        for portfolio in beta_estimates.keys():
            y = window[portfolio].dropna()
            X = window[['MP', 'UPR', 'UTS', 'UI', 'DEI']]*100
            
            # Proceed if both X and y have enough data points
            if len(X) == len(y) and len(X) >= 60:
                X = sm.add_constant(X)
                model = sm.OLS(y, X).fit()
                
                # Store the beta estimates
                beta_estimates[portfolio][year] = model.params[1:].values

    return beta_estimates

# Now we cross our fingers and hope that the function works
betas = estimate_betas(Assets_and_Macro_CRR, start_date, end_date)

In [57]:
# SANITY CHECK: Print the beta estimates for portfolio 'q5' and 'q10' for the first few years. If there is something obviously wrong it should be visible here. 
print("Beta estimates for portfolio 'q5':")
for year in range(pd.to_datetime(start_date).year + 6, pd.to_datetime(start_date).year + 8):
    print(f"Year {year}: {betas['q5'].get(year)}")

# Optionally, print another portfolio's beta estimates for the same years
print("\nBeta estimates for portfolio 'q10':")
for year in range(pd.to_datetime(start_date).year + 6, pd.to_datetime(start_date).year + 8):
    print(f"Year {year}: {betas['q10'].get(year)}")

Beta estimates for portfolio 'q5':
Year 1958: [ 0.01280434  0.00341176  0.0020279  -0.03575336  0.08953775]
Year 1959: [ 0.01009113  0.0035779   0.00191374 -0.02318621 -0.03655713]

Beta estimates for portfolio 'q10':
Year 1958: [ 0.01069057  0.00672404  0.00476033 -0.07035246  0.23751267]
Year 1959: [ 0.00863989  0.00866787  0.00581436 -0.04933504  0.13792684]


<div style="background-color: #f4f4f4; color: #333333; padding: 20px; border-radius: 10px; box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1); max-width: 3500px; margin: auto;">

## Step B: Cross-Sectional Regressions

We will now use the resulting estimates of exposure (betas) from Step A as the independent variables in cross-sectional regressions, one regression for each of the next 12 months, with asset returns for the month being the dependent variable: 

$$
\operatorname{E}\left[r_{p}\right]=\mu+\lambda_{1} \beta_{1 p}+\lambda_{2} \beta_{2 p}+\lambda_{3} \beta_{3 p}+\lambda_{4} \beta_{4 p}+\lambda_{5} \beta_{5 p}+e_{s},
$$

gives estimates of market prices of risk for each month: $\hat{\lambda}_{j}, j=1,5$.

Then we simply repeat the exercise for all the years in the sample, yielding for each macro variable a time series of estimates of its associated risk premium. 

</div>

In [58]:
def cross_sectional_regressions(data, betas, start_date, end_date):
    # Convert the string dates you picked to datetime objects
    start_date_dt = pd.to_datetime(start_date)
    end_date_dt = pd.to_datetime(end_date)
    
    # Derive the year range from the datetime objects
    start_year = start_date_dt.year
    end_year = end_date_dt.year
    
    # Prepare a structure to hold the lambda results for each factor
    lambda_results = {factor: [] for factor in ['MP', 'UPR', 'UTS', 'UI', 'DEI']}
    
    years = range(start_year, end_year + 1)  # Include the end year in the range
    
    for year in years:
        # Collect beta values for the year if available
        beta_values = {p: betas[p].get(year) for p in betas if betas[p].get(year) is not None}
        if not beta_values:  # If no beta data for the year, skip
            for factor in lambda_results.keys():
                lambda_results[factor].append(np.nan)
            continue
        
        # Prepare dataframes for regression
        beta_df = pd.DataFrame.from_dict(beta_values, orient='index', columns=['MP', 'UPR', 'UTS', 'UI', 'DEI'])
        returns_df = data[data.index.year == year][[f'q{q}' for q in range(10, 100, 5)]].sum()
        
        # Ensure we're only working with portfolios that have both returns and betas
        common_portfolios = beta_df.index.intersection(returns_df.index)
        beta_df = beta_df.loc[common_portfolios]
        y = returns_df.loc[common_portfolios]
        
        # Run the regression
        X = sm.add_constant(beta_df)
        model = sm.OLS(y, X).fit()
        
        # Store lambda values from regression coefficients
        for factor in ['MP', 'UPR', 'UTS', 'UI', 'DEI']:
            lambda_results[factor].append(model.params.get(factor, np.nan))
    
    # Create a DataFrame for the results spanning the defined years
    lambda_df = pd.DataFrame(lambda_results, index=np.arange(start_year, end_year + 1))
    return lambda_df

# Now, call the function and you will see the lambda values for each factor in each year
risk_premium_df = cross_sectional_regressions(Assets_and_Macro_CRR, betas, start_date, end_date).dropna()
print(risk_premium_df)

             MP        UPR        UTS        UI       DEI
1958  13.230635  -9.107881   3.072733 -1.515235 -0.907008
1959  12.818301 -13.646873  31.065042 -1.321037 -0.312856
1960 -32.146936 -22.409027  11.745448  0.166751  0.100640
1961 -16.869487 -18.883437  28.510449 -2.089983 -0.491605
1962  15.770853  14.202195 -10.176196 -0.610020 -0.519104
...         ...        ...        ...       ...       ...
2017  -0.375401 -11.217087  31.383276 -0.632877 -0.113887
2018   0.994000  -9.003911 -15.107930  0.330586  0.170646
2019  10.534570 -25.474881 -17.850361 -0.963677 -0.555210
2020  -3.014776 -37.870716  -8.392799  2.083865  2.565745
2021  15.674567 -35.620621  22.707636 -2.298934  0.924950

[64 rows x 5 columns]


The time-series means of these estimates are now tested to check if they are statistically different from zero. 

In [59]:
# Convert the string dates to datetime objects to extract the year
start_year = pd.to_datetime(start_date).year
end_year = pd.to_datetime(end_date).year

# Define the periods based on what you picked as start_date and end_date
periods = {
    f'{start_year}-{end_year}': (str(start_year), str(end_year)),
}

# Now we store the results in a dictionary
summary_table = {}

# Iterate over the periods, calculating means and t-stats for each factor
for period, (start_year, end_year) in periods.items():
    # Filtering the DataFrame for the current period
    df_period = risk_premium_df.loc[int(start_year):int(end_year)]
    
    # Calculating mean lambda values
    mean_values = df_period.mean()
    
    # Calculating t-statistics
    t_stats = mean_values / (df_period.std() / np.sqrt(df_period.count()))
    
    # Store the results in a DataFrame
    summary_table[period] = pd.DataFrame({
        'Lambda Mean': mean_values,
        'T-Stats': t_stats,
    }).round(2)

# Now we will spice up the display of the summary table
def display_summary_table(summary_table):
    for period, df in summary_table.items():
        display_html = df.style.set_table_attributes("style='display:inline'").set_caption(f"Period: {period}")\
                               .format("{:.2f}")\
                               .set_properties(**{'text-align': 'right', 'font-weight': 'bold'})\
                               .set_table_styles([dict(selector='th', props=[('text-align', 'center')])])
        display(display_html)

# Display the summary table
print("T-stats for the risk premia estimates are shown in parentheses")
display_summary_table(summary_table)

T-stats for the risk premia estimates are shown in parentheses


,Lambda Mean,T-Stats
MP,3.79,2.67
UPR,0.53,0.21
UTS,1.41,0.31
UI,-0.20,-0.60
DEI,0.06,0.51


<div style="background-color: #f4f4f4; color: #333333; padding: 20px; border-radius: 10px; box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1); max-width: 3500px; margin: auto;">

<p style="color: #4dc9f6; border-bottom: 2px solid #4dc9f6;"><strong>Summary</strong></p>

* The **positive risk premium on MP and UPR** signals that these are **risk-enhancing** factors. High level of MP and UPR are associated with *high* level of human capital wealth.

* The **negative risk premium on DEI, UI and UTS** signals that these are **insurance or hedging** factors. High level of DEI, UI and UTS are associated with *low* level of human capital wealth. 


</div>